In [1]:
from parcels import ParticleSet
from parcels import JITParticle
from parcels import AdvectionRK4_3D
from parcels import AdvectionRK4
from datetime import timedelta
import numpy as np
from parcels import FieldSet
from glob import glob
import xarray as xr

In [6]:
## parameters
## set the number of particles
num_particles = 10_000

## Set the location of the particles
channel_lon_bds = (0,4.5) # use inmutable objects
channel_lat_bds = (0+1,18-1)

## set the tracking time
days = 360
minutes = 20

## record the particles every timestep of
hours=4

In [3]:
##prepare the files
path1= '/gxfs_work/geomar/smomw662/FESOM_data/channel_interp'

ds_w = xr.open_mfdataset("/gxfs_work/geomar/smomw662/FESOM_data/channel_interp/w.196*.nc")
ds_v = xr.open_mfdataset("/gxfs_work/geomar/smomw662/FESOM_data/channel_interp/v.196*.nc")
ds_u = xr.open_mfdataset("/gxfs_work/geomar/smomw662/FESOM_data/channel_interp/u.196*.nc")

ds_u=ds_u.drop_vars('nz')
ds_v=ds_v.drop_vars('nz')
ds_w=ds_w.drop_vars('nz')

ds_uvw_grid= xr.Dataset({
    'U':ds_u.u,
    "V":ds_v.v,
    "W":ds_w.w,
})

## starting positions
lon_start = np.random.uniform(*channel_lon_bds,size=(num_particles,)) 
lat_start = np.random.uniform(*channel_lat_bds, size=(num_particles,))

In [4]:
ds_uvw_grid = ds_uvw_grid.compute()

In [5]:
## fieldset
fieldset = FieldSet.from_xarray_dataset(
    ds_uvw_grid.transpose('time','z','grid_lat','grid_lon'),
    variables={'U':"U", "V":"V", "W":"W"},
    dimensions={'lon':'grid_lon',
                'lat':'grid_lat',
                'depth':'z',
                'time':'time',
               },
    time_periodic=False,
    allow_time_extrapolation=False,
)

In [7]:
#halo
fieldset.add_constant("halo_west", fieldset.U.grid.lon[0])
fieldset.add_constant("halo_east", fieldset.U.grid.lon[-1])
fieldset.add_periodic_halo(zonal=True)

def periodicBC(particle,fielset,time):
    if particle.lon < fieldset.halo_west:
        particle_dlon += fieldset.halo_east - fieldset.halo_west
    elif particle.lon > fieldset.halo_east:
        particle_dlon -= fieldset.halo_east - fieldset.halo_west

In [8]:
## Time and depth initial conditios
time = np.repeat(ds_uvw_grid.time[0].data, num_particles)  # Assign the same time to all particles
depth = np.random.uniform(10,50, size=num_particles)  # Choose random depths

In [9]:
#Init particle set
pset = ParticleSet(
    fieldset=fieldset,
    pclass=JITParticle,
    lon = lon_start,
    lat = lat_start,
    depth=depth,
    time=time
) 

In [10]:
out_path = '/gxfs_work/geomar/smomw662/FESOMparcels_first/data/' #path to store the particle zarr
out_fn = 'Parcels_run_2'

output_file = pset.ParticleFile(name=out_path+out_fn, 
                                outputdt=timedelta(hours=hours))

In [ ]:
## Execute particles
pset.execute(
    [AdvectionRK4_3D,periodicBC],
    runtime=timedelta(days=days),
    dt=timedelta(minutes=minutes),
    output_file= output_file
)

INFO: Output files are stored in /gxfs_work/geomar/smomw662/FESOMparcels_first/data/Parcels_run.zarr.
  0%|▏                                | 144000.0/31104000.0 [02:20<6:59:27, 1230.17it/s]